# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


In [2]:
ws = Workspace.from_config()

# Choosing a name for experiment
experiment_name = 'automl-exp'

experiment=Experiment(ws, experiment_name)

## Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset "Heart Failure Prediction" contains 12 clinical features of 299 patients with heart failure and a target variable "DEATH EVENT" indicating if the patient deceased during the follow-up period (bool). Machine Learning can help detect and manage high-risk patients at an earlier stage.

To solve this binary classification problem, 12 features are used to predict possible death events with the help of an ML algorithm.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
key = "Heart Failure Prediction"
description_text = "Heart Failure Prediction DataSet for Udacity Project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("The dataset is loaded")

else:
        # Create Dataset and register it into Workspace
        example_data = "https://raw.githubusercontent.com/ruthussanketh/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

The dataset is loaded


In [5]:
df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Automl settings: 
1. experiment_timeout_minutes - Maximum amount of time the experiment can take. 20 minutes were set to save time. 
2. max_concurrent_iterations - Maximum number of parallel runs executed on a AmlCompute cluster. Because it should be less than or equal to the number of nodes (5) set when creating the compute cluster, it is set to 5. 
3. The primary metric is Under the Curve Weighted, **AUC_weighted**, to deal with class imbalance. 
4. AutoMLConfig: This is a binary classification task. 
5. The label inside the dataset which it to be predicted is "DEATH_EVENT". 
6. To save time and resources, the enable_early_stopping parameter is set to True.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Retrieving the best automl model
automl_best_run, fitted_automl_model = automl_run.get_output() 

run_dict = automl_best_run.get_details()

print("Best Run ID: ", run_dict["runId"])
print("---"*10)
print("AutoML best run AUC_weighted: ", automl_best_run.get_metrics(name="AUC_weighted"))
print("---"*10)
print("Metrics: ", automl_best_run.get_metrics())
print("---"*10)
print("Fitted model and its hyperparameters : ", fitted_automl_model.steps[-1])

Best Run ID:  AutoML_360fc922-4cfe-4f8a-aa6d-851af887945a_48
------------------------------
AutoML best run AUC_weighted:  {'AUC_weighted': 0.9215367227907274}
------------------------------
Metrics:  {'balanced_accuracy': 0.8263997217186576, 'precision_score_weighted': 0.8671542819362292, 'weighted_accuracy': 0.8745166166047295, 'AUC_weighted': 0.9215367227907274, 'average_precision_score_weighted': 0.9326414051101513, 'precision_score_micro': 0.8559770114942529, 'norm_macro_recall': 0.6527994434373152, 'AUC_micro': 0.9236981107147576, 'recall_score_weighted': 0.8559770114942529, 'log_loss': 0.3816592595041143, 'matthews_correlation': 0.6721443941147944, 'f1_score_weighted': 0.853329953772986, 'recall_score_macro': 0.8263997217186576, 'recall_score_micro': 0.8559770114942529, 'f1_score_micro': 0.8559770114942529, 'precision_score_macro': 0.847553467774056, 'average_precision_score_micro': 0.927577838218693, 'f1_score_macro': 0.82743389162527, 'average_precision_score_macro': 0.9099471

In [10]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model', model_path = './outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
automl_best_run.get_file_names()

# Downloading the scoring file 
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py' , 'score.py')

# Downloading conda environment 
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml' , 'env.yml')

In [12]:
from azureml.core import Model
# Creating the environment
env = Environment.get(workspace=ws, name="AzureML-AutoML")
# Creating an inference config
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

# Deploying the model as a web service to an Azure Container Instance (ACI)
service_name = 'hfprediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

webservice.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-06 15:04:08+00:00 Creating Container Registry if not exists.
2021-04-06 15:04:09+00:00 Registering the environment.
2021-04-06 15:04:11+00:00 Use the existing image.
2021-04-06 15:04:12+00:00 Generating deployment configuration.
2021-04-06 15:04:13+00:00 Submitting deployment to compute..
2021-04-06 15:04:19+00:00 Checking the status of deployment hfprediction..
2021-04-06 15:08:46+00:00 Checking the status of inference endpoint hfprediction.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
scoring_uri = "http://c27402a6-6dd0-4ce8-a9f4-ba6da94647c6.southcentralus.azurecontainer.io/score"

df = df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
    'data': df[0:2].to_dict(orient='records')
})


print(input_data)

{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}, {"age": 55.0, "anaemia": 0, "creatinine_phosphokinase": 7861, "diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 0, "platelets": 263358.03, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 0, "time": 6}]}


TODO: In the cell below, print the logs of the web service and delete the service

In [14]:
# Retrieve logs for this Webservice
webservice.get_logs()

'2021-04-06T15:08:40,975640700+00:00 - rsyslog/run \n2021-04-06T15:08:40,984272600+00:00 - gunicorn/run \n2021-04-06T15:08:40,988527200+00:00 - iot-server/run \n2021-04-06T15:08:41,049492000+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [15]:
# Delete service
webservice.delete()